# **Esercizio 5: task labeling con LLM**

In [1]:
!pip install transformers==4.40.2



In [2]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
def setup_model():
    print("Verificando disponibilità GPU...")
    gpu_available = torch.cuda.is_available()
    print(f"GPU disponibile: {gpu_available}")

    print("Caricamento del modello...")
    model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3.5-mini-instruct",
        device_map="auto",
        trust_remote_code=True
    )

    print("Caricamento del tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        "microsoft/Phi-3.5-mini-instruct",
        trust_remote_code=True
    )

    print("Creazione della pipeline...")
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        do_sample=True,
        max_new_tokens=500,
        temperature=0.2,
        top_p=1
    )

    return pipe


def generate_response(pipe, prompt):
    """
    Genera una risposta usando il modello
    """
    messages = [
        {"role": "user", "content": prompt}
    ]

    output = pipe(messages)

    generated_text = output[0]["generated_text"]


    if isinstance(generated_text, list):

        for message in generated_text:
            if isinstance(message, dict) and message.get("role") == "assistant":
                return message.get("content", "")
        return ""
    else:
        # If it's a string, remove the prompt if it's included
        if generated_text.startswith(prompt):
            generated_text = generated_text[len(prompt):].lstrip()
        return generated_text

# Laboratorio N. 5

Come attività pratica, si propone un esercizio di labeling su due sottotask (5.1 e 5.2):
 1.  Generare etichette per i topic del laboratorio n.4. 2.
 2. Indovinare le parole sulla base delle definizioni dei
laboratori n.2 e n.3.
Si raccomanda di utilizzare prompt ben progettati e ragionati, in maniera iterativa, selezionando bene i dati da includere nei prompt e in generale seguendo le strategie apprese in questo capitolo.

## **Lab 5.1**

### Funzioni di utilità

In [4]:
def get_topic_prompt(index, topic, weight):
  prompt = ""
  for word in topic:
    prompt = prompt + ", " + word + "(" + str(weight[topic.index(word)]) + ")"
  return """Topic: """ + str(index) +""": <INPUT>: """ + prompt

def get_labels_prompt(topics, weights):
  topics_prompt = ""
  for topic in topics:
    topics_prompt = topics_prompt + " " + get_topic_prompt(topics.index(topic), topic, weights[topics.index(topic)])


  labeling_prompt = """Make a liast of labels for some topics, the topic is a list of words more used in the document with the weight, greater is the weight mor is used the word in the document. The OUTPUT is the label of the given untitled topic. The INPUT is the topic.
       Example of input - output: Topic: 0 <INPUT>: generative(0.023), model(0.013), probabilistic(0.033), robot(0.019), task(0.022) Topic: 1 <INPUT>: student(0.023), book(0.013), lesson(0.033), teacher(0.019), result(0.022) <OUTPUT>: "School"
       ## the real input is:
       """ + topics_prompt
  return labeling_prompt

def get_label_prompt(topic, weight):
  labeling_prompt = """Make a label for a topic, the topic is a list of words more used in the document with the weight, greater is the weight mor is used the word in the document. The OUTPUT is the label of the given untitled topic. The INPUT is the topic.
     Example of input - output: <INPUT>: generative(0.023), model(0.013), probabilistic(0.033), robot(0.019), task(0.022) <OUTPUT>: "Artificial inteligence"
     ## the real input is:
       """ + get_topic_prompt(0, topic, weight)
  return labeling_prompt

def get_response( prompt: str, pipe ):
  labeling_response = generate_response(pipe, prompt)
  return labeling_response

### Codice

In [8]:
pipe = setup_model()

Verificando disponibilità GPU...
GPU disponibile: True
Caricamento del modello...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Caricamento del tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Creazione della pipeline...


In [ ]:
topics = [
  ["trump", "white", "black", "peaple", "donald"],
  ["russia", "russian", "trump", "fbi", "say"],
  ["korea", "china", "north", "say", "nuclear"],
  ["clinton", "sanders", "hillary", "campaign", "bernie"],
  ["immigration", "ban", "immigrant", "order", "court"],
  ["cruz", "trump", "rubio", "republican", "ted"],
  ["merkel", "european", "germany", "macron", "party"],
  ["muslim", "attack", "say", "police", "london"]
]
weights = [
  [0.041, 0.022, 0.021, 0.020, 0.018],
  [0.038, 0.033, 0.029, 0.026, 0.025],
  [0.049, 0.049, 0.046, 0.032, 0.028],
  [0.071, 0.048, 0.041, 0.030, 0.022],
  [0.040, 0.031, 0.030, 0.030, 0.028],
  [0.082, 0.039, 0.039, 0.034, 0.030],
  [0.052, 0.038, 0.038, 0.038, 0.030],
  [0.031, 0.022, 0.020, 0.019, 0.019]
]


get_response(get_label_prompt( topics[0] , weights[0]), pipe)
results = []

for topic in topics:
  response = get_response(get_label_prompt(topic, weights[topics.index(topic)]), pipe)
  results.append(response)

index = 1
for result in results:
  print("----- Topic " + str(index) + " -------- " )
  print(result)
  print("----------------------------------------")
  index += 1

print("")
print("---- With all topic -------")
all_results = get_response(get_labels_prompt(topics, weights), pipe)
print(all_results)




----- Topic 1 -------- 
 <OUTPUT>: "Political discourse and racial themes"

Explanation: The given words are related to political figures (Donald Trump) and racial demographics (white, black, people). The frequency of these words suggests that the topic revolves around discussions or themes involving politics, particularly in the context of racial issues or demographics. Hence, the label "Political discourse and racial themes" encapsulates the essence of the topic.
----------------------------------------
----- Topic 2 -------- 
 <OUTPUT>: "US-Russia Relations and Investigations"

Explanation: The given words are related to geopolitical events, particularly involving Russia, the United States, and investigative bodies like the FBI. The terms "Russia," "Russian," and "Trump" suggest a focus on US-Russia relations, possibly in the context of political or investigative matters. The presence of "FBI" indicates an investigative angle, and "say" could imply statements or comments from involv

----- Topic 1 --------

Political discourse and racial themes

- Explanation: The given words are related to political figures (Donald Trump) and racial demographics (white, black, people). The frequency of these words suggests that the topic revolves around discussions or themes involving politics, particularly in the context of racial issues or demographics. Hence, the label "Political discourse and racial themes" encapsulates the essence of the topic.
----------------------------------------
----- Topic 2 --------
 <OUTPUT>: "US-Russia Relations and Investigations"

- Explanation: The given words are related to geopolitical events, particularly involving Russia, the United States, and investigative bodies like the FBI. The terms "Russia," "Russian," and "Trump" suggest a focus on US-Russia relations, possibly in the context of political or investigative matters. The presence of "FBI" indicates an investigative angle, and "say" could imply statements or comments from involved parties. Combining these elements, the label "US-Russia Relations and Investigations" encapsulates the topic's essence, reflecting the weight and context of the words provided.
----------------------------------------
----- Topic 3 --------
 <OUTPUT>: "North Korea Nuclear Issues"

- Explanation: The given words suggest a topic related to geopolitical tensions, specifically focusing on North Korea's nuclear capabilities. "North Korea" and "nuclear" are central to the topic, while "China" might be involved as a regional player or influencer. The other terms like "say" could relate to statements or positions taken by involved parties. Thus, the label "North Korea Nuclear Issues" encapsulates the main theme of the topic based on the weighted words provided.
----------------------------------------
----- Topic 4 --------
 <OUTPUT>: "US Presidential Election Campaigns"

- Explanation: The given words are related to political figures (Clinton, Sanders, Hillary, Bernie) and the context of a campaign. This suggests that the topic is about the US Presidential Election Campaigns, where these individuals are significant players or subjects of discussion. The weights indicate the frequency or prominence of these terms in the document, reinforcing the political theme.
----------------------------------------
----- Topic 5 --------
 <OUTPUT>: "Immigration Policy and Legal Issues"

- Explanation: The given words are related to immigration, legal proceedings, and regulatory actions. The terms "immigration," "ban," "immigrant," "order," and "court" suggest a topic centered around immigration policy, legal challenges, and enforcement. Therefore, the label "Immigration Policy and Legal Issues" encapsulates the essence of the topic based on the weighted frequency of the mentioned words.
----------------------------------------
----- Topic 6 --------
 <OUTPUT>: "Political Discourse in the United States"

- Explanation: The given words are associated with prominent political figures (Cruz, Trump, Rubio) and political parties (Republican), along with a reference to Ted, possibly a political figure or context. The weights indicate their frequency or importance in the document. This suggests the topic revolves around political discussions, debates, or events in the United States.
----------------------------------------
----- Topic 7 --------
 <OUTPUT>: "German Politics and European Union"

- Explanation: The given words indicate a topic related to German politics and its influence within the European Union. 'Merkel' refers to Angela Merkel, a prominent German politician who served as the Chancellor of Germany. The term 'European' and 'Germany' suggest a focus on European affairs, while 'Macron' refers to Emmanuel Macron, the President of France, indicating a broader European context. 'Party' suggests political entities or groups involved. Thus, the label "German Politics and European Union" encapsulates the essence of the topic.
----------------------------------------
----- Topic 8 --------
 <OUTPUT>: "London Islamic Attacks and Discourse"

- Explanation: The given words suggest a topic related to events or discussions involving Muslims, attacks, and the police in London. The label "London Islamic Attacks and Discourse" encapsulates these elements, indicating a focus on incidents and conversations about Islamic-related attacks in the city of London. The weight of each word reflects its relative importance or frequency in the context of the topic.
----------------------------------------



---- With all topic -------

Topic: 0 - "Politics/Current Events"

Topic: 1 - "International Relations"

Topic: 2 - "Nuclear Issues/Security"

Topic: 3 - "Political Campaigns/Elections"

Topic: 4 - "Immigration Policy"

Topic: 5 - "US Politics/Partisan Debates"

Topic: 6 - "European Politics"

Topic: 7 - "Societal Issues/Religious Tensions"


Each label represents the overarching theme or subject matter that the words and their associated weights suggest in the given topics.

## **Lab 5.2**

In [5]:
def load_top_k_terms(json_path=None):

    if json_path is None:
        # Percorso di default relativo alla struttura del progetto
        json_path = "/content/sample_data/top_4_terms_in_definitions.json"

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            top_k_data = json.load(f)
        print(f"Caricati dati per {len(top_k_data)} concetti dal file JSON")
        return top_k_data
    except FileNotFoundError:
        print(f"File JSON non trovato: {json_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Errore nel parsing del file JSON: {json_path}")
        return {}

In [6]:
def content_to_form_labeling(pipe, prompt_data, word):
    list_of_top_k_terms = prompt_data[word]["terms"]
    definition = prompt_data[word]["definition"]
    terms = [(item["term"], item["count"]) for item in list_of_top_k_terms]
    labeling_prompt = f"""
    Un insieme di definizioni sono associate ad un termine. Da queste definizioni vengono estratte le parole più frequenti.
    Esegui un compito di labeling trovando il termine associato alle definizioni avendo a disposizione la lista delle parole più frequenti nelle definizioni.
    INPUT: [{terms}] lista di tuple (parola, conteggio) delle  parole più frequenti presenti nell'insieme di definizioni del termine da trovare. Viene inoltre fornita una definizione di esempio: '{definition}'
    OUTPUT: il termine è rappresentato da una singola parola (sostantivo comune al singolare).
    Ad esempio se le parole più frequenti sono [(gioco, 40), (bambini, 33), (giardino, 22), (divertimento, 10)] il termine è altalena.
    Per svolgere il compito procedi per passi:
    1) osserva la lista di (parole, frequenze) data in input e deall'esempio di definizione fornita in input.
    2) estrai informazioni da esse, trovando delle relazioni tra queste parole, aiutandoti a capire a quale termine quella definizione è associata.
    3) utilizzare tutte le informazioni per risalire al termine.
    """
    response = generate_response(pipe, labeling_prompt)
    return response



def content_to_form_labeling_abstract_term(pipe, prompt_data, word):
    list_of_top_k_terms = prompt_data[word]["terms"]
    definition = prompt_data[word]["definition"]
    terms = [(item["term"], item["count"]) for item in list_of_top_k_terms]
    labeling_prompt = f"""
    Un insieme di definizioni sono associate ad un termine. Da queste definizioni vengono estratte le parole più frequenti.
    Esegui un compito di labeling trovando il termine associato alle definizioni avendo a disposizione la lista delle parole più frequenti nelle definizioni.
    INPUT: [{terms}] lista di tuple (parola, conteggio) delle parole più frequenti presenti nell'insieme di definizioni del termine da trovare. Viene inoltre fornita una definizione di esempio: '{definition}'
    OUTPUT: il termine rappresentato da una singola parola (sostantivo comune al singolare).

    Per svolgere il compito procedi per passi:
    1) Lavora in italiano.
    2) osserva la lista di (parole, frequenze) data in input e deall'esempio di definizione fornita in input.
    3) estrai informazioni da esse, trovando delle relazioni tra queste parole, aiutandoti a capire a quale termine quella definizione è associata.
    4) utilizzare tutte le informazioni per risalire al termine astratto. Scegli la forma più comune e generale in italiano.
    """
    response = generate_response(pipe, labeling_prompt)
    return response


In [9]:
if __name__ == "__main__":
    #pipe = setup_model()

    top_k_terms = load_top_k_terms()

    labeling_response = content_to_form_labeling(pipe, top_k_terms, 'Pantalone')
    print("Risultato del labeling:")
    print(labeling_response)

Caricati dati per 4 concetti dal file JSON


Risultato del labeling:
 Basato sulle informazioni fornite e sulla definizione di esempio, il termine associato alle parole più frequenti [(gamba, 26), (indumento, 25), (parte, 17), (corpo, 15)] è 'gambale'.

Ecco il ragionamento passo dopo passo:

1) Osserva la lista di (parole, frequenze): gamba (26), indumento (25), parte (17), corpo (15).
2) Analizza la definizione di esempio: 'indumento per coprire le gambe'. Qui, 'indumento' e 'gamba' sono le parole chiave che descrivono il concetto.
3) Relaziona le parole più frequenti con la definizione di esempio. 'Gamba' e 'indumento' sono presenti sia nella definizione di esempio che nella lista delle parole più frequenti.
4) Dato che 'gamba' è un sostantivo comune al singolare e è presente sia nella definizione di esempio che nella lista delle parole più frequenti, si può dedurre che il termine è 'gambale'.

Quindi, l'OUTPUT è: il termine è 'gambale'.


Caricati dati per 4 concetti dal file JSON

WARNING:transformers_modules.microsoft.Phi-3.5-mini-instruct.3145e03a9fd4cdd7cd953c34d9bbf7ad606122ca.modeling_phi3:You are not running the flash-attention implementation, expect numerical differences.

Risultato del labeling:
 Basato sulle informazioni fornite e sulla definizione di esempio, il termine associato alle parole più frequenti [(gamba, 26), (indumento, 25), (parte, 17), (corpo, 15)] è 'gambale'.

Ecco il ragionamento passo dopo passo:

1) Osserva la lista di (parole, frequenze): gamba (26), indumento (25), parte (17), corpo (15).
2) Analizza la definizione di esempio: 'indumento per coprire le gambe'. Qui, 'indumento' e 'gamba' sono le parole chiave che descrivono il concetto.
3) Relaziona le parole più frequenti con la definizione di esempio. 'Gamba' e 'indumento' sono presenti sia nella definizione di esempio che nella lista delle parole più frequenti.
4) Dato che 'gamba' è un sostantivo comune al singolare e è presente sia nella definizione di esempio che nella lista delle parole più frequenti, si può dedurre che il termine è 'gambale'.

Quindi, l'OUTPUT è: il termine è 'gambale'.


In [ ]:
if __name__ == "__main__":
    #pipe = setup_model()

    top_k_terms = load_top_k_terms()

    labeling_response = content_to_form_labeling(pipe, top_k_terms, 'Microscopio')
    print("Risultato del labeling:")
    print(labeling_response)

Caricati dati per 4 concetti dal file JSON


Risultato del labeling:
 Basato sulle informazioni fornite e sull'esempio, il termine associato alle parole più frequenti [(strumento, 32), (oggetto, 21), (piccolo, 15), (scientifico, 14)] e alla definizione 'Strumento che permette di ingrandire oggetti ed elementi fino a una scala impossibile da raggiungere ad occhio nudo' è probabilmente 'Microscopio'.

Ecco il ragionamento passo dopo passo:

1) Osserva la lista di parole più frequenti e la definizione:'strumento', 'oggetto', 'piccolo','scientifico'. Queste parole si riferiscono a un dispositivo utilizzato per ingrandire oggetti, che è un aspetto scientifico.

2) Relaziona le parole con la definizione:'strumento' e'scientifico' suggeriscono che il termine è un dispositivo utilizzato per scopi scientifici. 'oggetto' e 'piccolo' suggeriscono che questo dispositivo viene utilizzato per ingrandire oggetti piccoli.

3) Utilizza queste informazioni per risalire al termine: Considerando le parole e il loro contesto nella definizione, il ter

Etichettatura per "Microscopio":

Caricati dati per 4 concetti dal file JSON

WARNING:transformers_modules.microsoft.Phi-3.5-mini-instruct.3145e03a9fd4cdd7cd953c34d9bbf7ad606122ca.modeling_phi3:You are not running the flash-attention implementation, expect numerical differences.

Risultato del labeling:
 Basato sulle informazioni fornite e sull'esempio, il termine associato alle parole più frequenti [(strumento, 32), (oggetto, 21), (piccolo, 15), (scientifico, 14)] e alla definizione 'Strumento che permette di ingrandire oggetti ed elementi fino a una scala impossibile da raggiungere ad occhio nudo' è probabilmente 'Microscopio'.

Ecco il ragionamento passo dopo passo:

1) Osserva la lista di parole più frequenti e la definizione:'strumento', 'oggetto', 'piccolo','scientifico'. Queste parole si riferiscono a un dispositivo utilizzato per ingrandire oggetti, che è un aspetto scientifico.

2) Relaziona le parole con la definizione:'strumento' e'scientifico' suggeriscono che il termine è un dispositivo utilizzato per scopi scientifici. 'oggetto' e 'piccolo' suggeriscono che questo dispositivo viene utilizzato per ingrandire oggetti piccoli.

3) Utilizza queste informazioni per risalire al termine: Considerando le parole e il loro contesto nella definizione, il termine che meglio si adatta è 'Microscopio'. Un microscopio è uno strumento scientifico utilizzato per ingrandire oggetti piccoli al di là della portata dell'occhio umano.

Quindi, l'output è: Microscopio.


In [ ]:
if __name__ == "__main__":
    #pipe = setup_model()

    top_k_terms = load_top_k_terms()

    labeling_response = content_to_form_labeling_abstract_term(pipe, top_k_terms, 'Euristica')
    print("Risultato del labeling:")
    print(labeling_response)

Caricati dati per 4 concetti dal file JSON


Risultato del labeling:
 Basato sulle informazioni fornite e sulla relazione tra le parole più frequenti ('ricerca', 'problema','strategia', 'utilizzare') e la definizione di esempio ('regola che permette di approssimare una soluzione ottima a un problema'), il termine astratto associato a queste definizioni è probabilmente'metodo'.

Spiegazione:
- 'ricerca' e 'problema' implicano l'indagine o la soluzione di qualcosa.
-'strategia' suggerisce un piano o un approccio per raggiungere un obiettivo.
- 'utilizzare' indica l'applicazione di qualcosa.

Unendo queste idee,'metodo' emerge come termine astratto che comprende l'idea di un approccio strutturato (strategia), utilizzato per risolvere un problema (ricerca e problema) e applicato in un contesto specifico (utilizzare).

Quindi, il termine associato alle definizioni e alla definizione di esempio è'metodo'. In italiano, la forma più comune e generale di questo termine è'metodo' (sostantivo comune al singolare).

OUTPUT: metodo


Etichettatura per Euristica:

Caricati dati per 4 concetti dal file JSON

WARNING:transformers_modules.microsoft.Phi-3.5-mini-instruct.3145e03a9fd4cdd7cd953c34d9bbf7ad606122ca.modeling_phi3:You are not running the flash-attention implementation, expect numerical differences.

Risultato del labeling:
 Basato sulle informazioni fornite e sulla relazione tra le parole più frequenti ('ricerca', 'problema','strategia', 'utilizzare') e la definizione di esempio ('regola che permette di approssimare una soluzione ottima a un problema'), il termine astratto associato a queste definizioni è probabilmente'metodo'.

Spiegazione:
- 'ricerca' e 'problema' implicano l'indagine o la soluzione di qualcosa.
-'strategia' suggerisce un piano o un approccio per raggiungere un obiettivo.
- 'utilizzare' indica l'applicazione di qualcosa.

Unendo queste idee,'metodo' emerge come termine astratto che comprende l'idea di un approccio strutturato (strategia), utilizzato per risolvere un problema (ricerca e problema) e applicato in un contesto specifico (utilizzare).

Quindi, il termine associato alle definizioni e alla definizione di esempio è'metodo'. In italiano, la forma più comune e generale di questo termine è'metodo' (sostantivo comune al singolare).

OUTPUT: metodo


In [ ]:
if __name__ == "__main__":
    #pipe = setup_model()

    top_k_terms = load_top_k_terms()

    labeling_response = content_to_form_labeling_abstract_term(pipe, top_k_terms, 'Pericolo')
    print("Risultato del labeling:")
    print(labeling_response)

Caricati dati per 4 concetti dal file JSON


Risultato del labeling:
 Basato sulle informazioni fornite e sulla relazione tra le parole più frequenti ('situazione', 'persona', 'incolumità', 'il'), insieme alla definizione di esempio ('Percezione umana innescata da un evento che mina la sicurezza del soggetto'), il termine astratto che meglio si adatta è 'pericolo'.

Questo perché:
-'situazione' si riferisce a una condizione o stato particolare.
- 'persona' può essere il soggetto di tale situazione.
- 'incolumità' è legata alla sicurezza o alla protezione della persona.
- 'il' è un articolo che introduce il concetto di pericolo.

Quindi, il termine astratto che collega tutte queste parole e corrisponde alla definizione data è 'pericolo'. In italiano, la forma più comune e generale di questo termine è 'pericolo'.

OUTPUT: pericolo


Caricati dati per 4 concetti dal file JSON

WARNING:transformers_modules.microsoft.Phi-3.5-mini-instruct.3145e03a9fd4cdd7cd953c34d9bbf7ad606122ca.modeling_phi3:You are not running the flash-attention implementation, expect numerical differences.

Risultato del labeling:
 Basato sulle informazioni fornite e sulla relazione tra le parole più frequenti ('situazione', 'persona', 'incolumità', 'il'), insieme alla definizione di esempio ('Percezione umana innescata da un evento che mina la sicurezza del soggetto'), il termine astratto che meglio si adatta è 'pericolo'.

Questo perché:
-'situazione' si riferisce a una condizione o stato particolare.
- 'persona' può essere il soggetto di tale situazione.
- 'incolumità' è legata alla sicurezza o alla protezione della persona.
- 'il' è un articolo che introduce il concetto di pericolo.

Quindi, il termine astratto che collega tutte queste parole e corrisponde alla definizione data è 'pericolo'. In italiano, la forma più comune e generale di questo termine è 'pericolo'.

OUTPUT: pericolo
